In [ ]:
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_chroma import Chroma
#from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate

In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Get the secret key using getenv function
secret_key = os.getenv('OPENAI_API_KEY')

model = init_chat_model("gpt-4o-mini",model_provider="openai", api_key = secret_key)

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
# Index chunks
#vector_store = InMemoryVectorStore(embeddings)
vector_store = Chroma(
    collection_name="langchain_docs",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [ ]:
# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [ ]:
# print how many documents were loaded
print(f"Loaded {len(docs)} documents.")

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
vector_store.add_documents(documents=all_splits)

In [ ]:
print(f"Indexed {vector_store._collection.count()} chunks.")

In [ ]:
# Define prompt for question-answering
# N.B. for non-US LangSmith endpoints, you may need to specify
# api_url="https://api.smith.langchain.com" in hub.pull.
#prompt = hub.pull("rlm/rag-prompt")

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that answers questions based on provided context."),
    ("user", """Answer the following question based only on the provided context. Keep the answer concise.

Context: {context}

Question: {question}

Answer: """)
])

In [ ]:
# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = model.invoke(messages)
    return {"answer": response.content}

In [ ]:
# Compile application and test
graph_graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_graph_builder.add_edge(START, "retrieve")
graph = graph_graph_builder.compile()

In [ ]:
response = graph.invoke({"question": "What is Task Decomposition?"})
print(response["answer"])